In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import platform

# OS 확인
if platform.system() == 'Windows':
    font_name = 'Malgun Gothic'  # 윈도우 기본 한글 폰트
elif platform.system() == 'Darwin':  # Mac
    font_name = 'AppleGothic'
else:
    font_name = 'NanumGothic'  # Linux 환경이면 설치 필요

plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 부호 깨짐 방지


In [9]:


# ✅ 파일 불러오기
# 1인 가구 비율
df_single_household = pd.read_csv("1인가구비율/1인가구비율_시도_시_군_구__20250721150446.csv", encoding="cp949")

# 경제활동참가율 (월별)
econ_df = pd.read_csv("경제활동인구/경제활동인구비율.csv", encoding="cp949")

# ✅ 1인 가구 비율 데이터 전처리
df_single_household_cleaned = df_single_household.iloc[1:].copy()
df_single_household_cleaned = df_single_household_cleaned.rename(columns={"행정구역별": "시도"})
year_cols = [str(y) for y in range(2015, 2024)]
df_single_ratio = df_single_household_cleaned[["시도"] + year_cols].copy()

for col in year_cols:
    df_single_ratio[col] = pd.to_numeric(df_single_ratio[col], errors="coerce")

df_single_ratio_melted = df_single_ratio.melt(id_vars="시도", var_name="연도", value_name="1인 가구 비율")
df_single_ratio_melted["연도"] = df_single_ratio_melted["연도"].astype(int)

# ✅ 경제활동참가율 전처리 (연도별 평균)
econ_df = econ_df[(econ_df["항목"] == "경제활동참가율[%]") & (econ_df["성별"] == "계")]
econ_df = econ_df.rename(columns={"시도별": "시도"})

value_cols = [col for col in econ_df.columns if "월" in col]
econ_long = econ_df.melt(id_vars=["시도"], value_vars=value_cols, var_name="월표기", value_name="경제활동참가율")

econ_long["월표기"] = econ_long["월표기"].str.replace(" 월", "")
econ_long["월기준"] = pd.to_datetime(econ_long["월표기"], format="%Y.%m")
econ_long["연도"] = econ_long["월기준"].dt.year

econ_yearly = econ_long.groupby(["시도", "연도"])["경제활동참가율"].mean().reset_index()

# ✅ 지역명 통일
region_map = {
    "서울특별시": "서울", "부산광역시": "부산", "대구광역시": "대구",
    "인천광역시": "인천", "광주광역시": "광주", "대전광역시": "대전",
    "울산광역시": "울산", "세종특별자치시": "세종", "경기도": "경기",
    "강원도": "강원", "충청북도": "충북", "충청남도": "충남",
    "전라북도": "전북", "전라남도": "전남", "경상북도": "경북",
    "경상남도": "경남", "제주도": "제주", "전국": "전국"
}
df_single_ratio_melted["시도"] = df_single_ratio_melted["시도"].replace(region_map)
econ_yearly["시도"] = econ_yearly["시도"].replace(region_map)

# ✅ 병합
df_merged = pd.merge(df_single_ratio_melted, econ_yearly, on=["시도", "연도"], how="inner")

# ✅ 시도별 상관계수 계산
correlation_results = []
for region in df_merged["시도"].unique():
    temp = df_merged[df_merged["시도"] == region]
    if len(temp) >= 3:
        corr, p = pearsonr(temp["1인 가구 비율"], temp["경제활동참가율"])
        correlation_results.append({
            "시도": region,
            "상관계수": corr,
            "p값": p
        })

df_corr = pd.DataFrame(correlation_results).sort_values("상관계수")

# ✅ 시각화
plt.figure(figsize=(12, 6))
sns.barplot(data=df_corr, x="시도", y="상관계수", palette="coolwarm")
plt.axhline(0, color="gray", linestyle="--")
plt.title("시도별 1인 가구 비율 vs 경제활동참가율 상관관계")
plt.ylabel("피어슨 상관계수")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ✅ 결과 확인
print(df_corr)


FileNotFoundError: [Errno 2] No such file or directory: '1인가구비율/1인가구비율_시도_시_군_구__20250721150446.csv'